# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

C:\Users\deon\Anaconda3\envs\learn-env\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [2]:
df = pd.read_json('./News_Category_Dataset_v2/News_Category_Dataset_v2.json', lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
authors              200853 non-null object
category             200853 non-null object
date                 200853 non-null datetime64[ns]
headline             200853 non-null object
link                 200853 non-null object
short_description    200853 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [4]:
df.columns

Index(['authors', 'category', 'date', 'headline', 'link', 'short_description'], dtype='object')

In [9]:
df['combined_text'] = df['headline']+' '+ df['short_description']
df['combined_text'][:10]

0    There Were 2 Mass Shootings In Texas Last Week...
1    Will Smith Joins Diplo And Nicky Jam For The 2...
2    Hugh Grant Marries For The First Time At Age 5...
3    Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4    Julianna Margulies Uses Donald Trump Poop Bags...
5    Morgan Freeman 'Devastated' That Sexual Harass...
6    Donald Trump Is Lovin' New McDonald's Jingle I...
7    What To Watch On Amazon Prime That’s New This ...
8    Mike Myers Reveals He'd 'Like To' Do A Fourth ...
9    What To Watch On Hulu That’s New This Week You...
Name: combined_text, dtype: object

In [10]:
data = df['combined_text'].map(word_tokenize)  # Apply word_tokenize to the text data in the combined variable.

Inspect the first 5 items in `data` to see how everything looks. 

In [11]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [12]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [13]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55566062, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [14]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [15]:
wv.most_similar('Texas')

[('Pennsylvania', 0.808169960975647),
 ('Florida', 0.7900899648666382),
 ('Louisiana', 0.7866576313972473),
 ('Georgia', 0.783429741859436),
 ('Oklahoma', 0.7792911529541016),
 ('Maryland', 0.778390645980835),
 ('Oregon', 0.7783544063568115),
 ('Arizona', 0.7772021293640137),
 ('Michigan', 0.7754590511322021),
 ('Ohio', 0.7744909524917603)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [16]:
wv.most_similar(negative='Texas')

[('Admon', 0.44293397665023804),
 ('Slammers', 0.39082351326942444),
 ('Drifters', 0.3815459609031677),
 ('SayHerName', 0.3785625696182251),
 ('Headstrong', 0.3731805086135864),
 ('Millennial-focused', 0.37242981791496277),
 ('Objection', 0.3634606897830963),
 ('Animal-rescue', 0.3633665442466736),
 ('Ex-Lions', 0.36092010140419006),
 ('Merabet', 0.3607962727546692)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [17]:
wv['Texas']

array([ 0.7213436 ,  1.4458139 , -1.8513465 , -0.3923439 , -0.16758196,
       -1.5270059 ,  0.92138934, -0.5562434 ,  1.5418217 , -0.294683  ,
        0.09119697, -0.6359781 ,  0.67346644, -1.0730754 , -2.3644903 ,
       -0.2278404 ,  0.07594555, -0.4319374 ,  0.37266546, -1.7552137 ,
        1.3175029 ,  1.926323  ,  1.7556633 ,  0.48825327,  1.5084534 ,
        0.9835283 , -0.9547011 ,  0.7301214 , -1.4485009 , -2.3229377 ,
       -0.07760558, -0.39772302,  0.46248484,  0.47695616, -2.1395166 ,
       -0.4144884 ,  0.35679182,  0.88310695, -1.0574138 ,  1.4772708 ,
        2.720981  , -0.2976036 , -0.2592541 ,  0.54021966,  0.7033883 ,
        0.21609233,  1.0153476 , -0.5897411 ,  1.3625549 ,  2.9317155 ,
        1.1890391 ,  1.3958496 , -0.676533  ,  3.872808  , -0.993071  ,
        1.6290973 ,  0.17427729, -0.389137  ,  0.08720312, -0.73208356,
       -1.8469925 ,  2.241312  , -0.39929256, -0.37628895, -1.4862521 ,
        0.33849925, -0.94699246, -1.7598739 ,  1.3321452 , -0.06

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [18]:
wv.vectors

array([[ 0.37431675,  0.23691285,  0.6577477 , ...,  1.9088193 ,
        -0.30689016,  1.131454  ],
       [-1.1383442 , -0.8214849 , -1.8259583 , ...,  0.71594906,
        -1.2852267 ,  1.0410022 ],
       [-0.9610882 , -0.1331229 ,  0.05612982, ...,  2.8642905 ,
         1.0967366 ,  1.6912628 ],
       ...,
       [ 0.05188506,  0.01371515,  0.03813282, ..., -0.11715858,
        -0.00378516,  0.01993968],
       [ 0.02433814,  0.04366071,  0.00361138, ..., -0.0074914 ,
         0.03485823, -0.05515237],
       [ 0.05084294,  0.02455942,  0.05483422, ..., -0.02660022,
        -0.00851207, -0.01284796]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [19]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('crown', 0.6164964437484741),
 ('queen', 0.5923128724098206),
 ('princess', 0.5879024863243103),
 ('symbol', 0.581986129283905),
 ('reminiscent', 0.5678521990776062),
 ('title', 0.5599974989891052),
 ('coin', 0.5577843189239502),
 ('shade', 0.5548304319381714),
 ('darling', 0.5539197325706482),
 ('revival', 0.5535280704498291)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!